*mounting to google drive*

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*install library prerequisites*

In [43]:
!pip install numpy scikit-learn pillow tqdm

#Imports

In [44]:
import os
import pickle
import numpy as np
import tensorflow as tf
from google.colab import drive
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, img_to_array,
                                               load_img)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

#Configuration

In [45]:
# --------------------- Configuration ---------------------
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
DATA_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data"
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
MODEL_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/models"
LABELS_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data/labels"
# --------------------- Configuration ---------------------

# script for finding the folders in drive

In [46]:
project_dir = '/content/drive/MyDrive/ASL/ASL_to_Text_Project'
if os.path.exists(project_dir):
  print(f"\nContents of {project_dir}:")
  print(os.listdir(project_dir))
else:
  print(f"Error: Directory not found: {project_dir}")
  print("Please ensure the path is correct and the drive is mounted.")

Error: Directory not found: /content/drive/MyDrive/ASL/ASL_to_Text_Project
Please ensure the path is correct and the drive is mounted.


#Directory tweaks

In [47]:
# Create directories if they don't exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LABELS_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

#mobilenet model Creation

In [48]:
def create_mobilenetv2_model(num_classes, input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """Enhanced MobileNetV2 model with improved anti-overfitting techniques"""
    # Base model configuration
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Selectively unfreeze more layers for fine-tuning
    for layer in base_model.layers[:-30]:
        layer.trainable = False
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Enhanced regularization
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(
        512,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.005),  # Slightly adjusted regularization
        activity_regularizer=regularizers.l1(0.0001)
    )(x)
    x = tf.keras.layers.Dropout(0.6)(x)  # Increased dropout

    # Output layer with stronger regularization
    outputs = tf.keras.layers.Dense(
        num_classes,
        activation='softmax',
        kernel_regularizer=regularizers.l1_l2(l1=0.003, l2=0.01)
    )(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
    return model

#Loading and Preprocessing Data

In [49]:
# Load and preprocess data
def load_data(data_dir, allowed_extensions=('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
    """Loads images and their corresponding labels."""
    images = []
    labels = []
    for label in os.listdir(data_dir):
        if label.startswith('.'):
            continue
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                if img_name.startswith('.'):
                    continue
                if not img_name.lower().endswith(allowed_extensions):
                    print(f"Skipping unsupported file: {img_name}")
                    continue
                try:
                    img_path = os.path.join(label_dir, img_name)
                    img = Image.open(img_path)
                    img.verify()
                    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                    img_array = img_to_array(img)
                    images.append(img_array)
                    labels.append(label)
                except (IOError, UnidentifiedImageError) as e:
                    print(f"Error loading image {img_name}: {e}")
                    continue
    return np.array(images), np.array(labels)

#training history

In [50]:
def plot_training_history(history):
    """Plot training history with improved styling."""
    sns.set_theme()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot loss
    ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
    ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax1.set_title('Training and Validation Loss', fontsize=12, pad=15)
    ax1.set_xlabel('Epoch', fontsize=10)
    ax1.set_ylabel('Loss', fontsize=10)
    ax1.legend(frameon=True)
    ax1.grid(True, linestyle='--', alpha=0.7)

    # Plot accuracy
    ax2.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax2.set_title('Training and Validation Accuracy', fontsize=12, pad=15)
    ax2.set_xlabel('Epoch', fontsize=10)
    ax2.set_ylabel('Accuracy', fontsize=10)
    ax2.legend(frameon=True)
    ax2.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    return fig

def plot_confusion_matrix(y_true, y_pred, classes):
    """Plot confusion matrix with improved visualization."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix', fontsize=12, pad=15)
    plt.xlabel('Predicted Label', fontsize=10)
    plt.ylabel('True Label', fontsize=10)
    plt.tight_layout()
    return plt.gcf()

#learning rate scheduler

In [51]:
def learning_rate_scheduler(epoch, lr):
    """Learning rate scheduler function.

    Args:
        epoch (int): Current epoch number.
        lr (float): Current learning rate.

    Returns:
        float: New learning rate.
    """
    if epoch < 10:
        return float(lr)  # Explicitly convert to float
    else:
        return float(lr * np.exp(-0.1))

# analyze model performance / calculations

In [52]:
def analyze_model_performance(model, history, X_test, y_test, le, X_train, EPOCHS):
    """Analyze and visualize model performance."""
    # Calculate total images trained
    total_images_trained = len(X_train) * EPOCHS
    print(f"\nTraining Summary:")
    print(f"Total Images Trained On: {total_images_trained:,}")
    print(f"Number of Epochs: {EPOCHS}")
    print(f"Training Set Size: {len(X_train):,} images")
    print(f"Test Set Size: {len(X_test):,} images")

    # Plot training history
    history_fig = plot_training_history(history)
    history_fig.savefig(os.path.join(MODEL_DIR, 'training_history.png'))

    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Plot confusion matrix
    cm_fig = plot_confusion_matrix(y_test_classes, y_pred_classes, le.classes_)
    cm_fig.savefig(os.path.join(MODEL_DIR, 'confusion_matrix.png'))

    # Generate classification report
    report = classification_report(y_test_classes, y_pred_classes,
                                 target_names=le.classes_,
                                 output_dict=True)

    # Save training metrics and classification report
    with open(os.path.join(MODEL_DIR, 'training_report.txt'), 'w') as f:
        f.write("Training Metrics:\n")
        f.write(f"Total Images Trained On: {total_images_trained:,}\n")
        f.write(f"Number of Epochs: {EPOCHS}\n")
        f.write(f"Training Set Size: {len(X_train):,} images\n")
        f.write(f"Test Set Size: {len(X_test):,} images\n\n")
        f.write("Classification Report:\n")
        for label in report:
            if label not in ['accuracy', 'macro avg', 'weighted avg']:
                f.write(f"\nClass: {label}\n")
                f.write(f"Precision: {report[label]['precision']:.3f}\n")
                f.write(f"Recall: {report[label]['recall']:.3f}\n")
                f.write(f"F1-Score: {report[label]['f1-score']:.3f}\n")
                f.write(f"Support: {report[label]['support']}\n")

    return report

# **MAIN EXECUTION**

#load ang preprocess data

In [53]:
# Load and preprocess data
X, y = load_data(IMAGES_DIR)
print(f"Number of images loaded: {len(X)}")
X = X / 255.0

Number of images loaded: 8153


#Encode Labels

In [54]:
# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print(f"Number of Classes : {num_classes}")

Number of Classes : 27


#Split Data

In [55]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

#Convert to categorical

In [56]:
# Convert to categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

#Model Creation and Compilation

In [57]:

# Create and compile model
model = create_mobilenetv2_model(num_classes)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

#Data Augmentation

In [58]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.15,
    shear_range=0.1,
    fill_mode="nearest",
    brightness_range=[0.9, 1.1]
)

#Early Stopping

In [59]:
# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Model Trainig

In [ ]:
# Update the training process in the main script
lr_schedule = tf.keras.callbacks.LearningRateScheduler(learning_rate_scheduler)


# Flatten y_train before applying np.unique to get unique class labels
unique_classes = np.unique(y_train.flatten())
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train.flatten())

# Convert the class weights to a dictionary
class_weight_dict = dict(enumerate(class_weights))

# Add ReduceLROnPlateau callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# Modify model.fit() callbacks to include new learning rate management
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(MODEL_DIR, "asl_mobilenetv2_model_{epoch:02d}_{val_accuracy:.2f}.keras"),
            monitor="val_accuracy",
            save_best_only=True,
            mode="max",
        ),
        early_stopping,
        lr_schedule,
        reduce_lr
    ],
    class_weight=class_weight_dict
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


203/203 ━━━━━━━━━━━━━━━━━━━━ 98s 404ms/step - accuracy: 0.0622 - loss: 12.0031 - val_accuracy: 0.0429 - val_loss: 10.8760 - learning_rate: 1.0000e-04
Epoch 2/30
  1/203 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.0625 - loss: 10.1060

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0625 - loss: 10.1060 - val_accuracy: 0.0423 - val_loss: 10.8699 - learning_rate: 1.0000e-04
Epoch 3/30
144/203 ━━━━━━━━━━━━━━━━━━━━ 21s 358ms/step - accuracy: 0.1464 - loss: 10.0686

#Dataset Evaluation

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

#Save Model

In [ ]:
# Save the entire model
model.save(os.path.join(MODEL_DIR, "asl_mobilenetv2_model.h5"))

#Save label encoder

In [ ]:
# Save the label encoder
with open(os.path.join(LABELS_DIR, 'cnn_label_encoder.pkl'), 'wb') as f:
    pickle.dump(le, f)

# Plot Training and Validation curves

In [ ]:
# Analyze model performance
print("\nAnalyzing model performance...")
performance_report = analyze_model_performance(model, history, X_test, y_test, le, X_train, EPOCHS)
print("\nAnalysis complete! Check the model directory for visualization plots and detailed reports.")